In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from collections import Counter
df = pd.read_csv('../datasets/movie_data.csv', encoding='utf-8')
df

2024-08-09 02:09:17.507279: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 02:09:17.507731: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 02:09:17.509557: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 02:09:17.515330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 02:09:17.525185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


In [2]:
# 데이터셋 객체 생성
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

# 데이터셋 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


I0000 00:00:1723136958.889031  274310 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-09 02:09:18.889367: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-08-09 02:09:18.950135: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(target.shape[0], reshuffle_each_iteration=False)

ds_raw = ds_raw.take(100)
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'Alright, let me break it down for ya... Haggard is' 1
b'I was impressed by the beautiful photography in th' 1
b"I've just watched Fingersmith, and I'm stunned to " 0


2024-08-09 02:09:19.016742: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
import myprocess  # 'myprocess.py' 파일을 import합니다.

preprocessor = myprocess.PreProcess(ds_raw, 100)

In [15]:
batch_size = 10

train_data, valid_data, test_data = preprocessor.get_datas(batch_size, train_size=0.4, valid_size = 0.1, test_size=0.5)

2024-08-09 02:12:57.420960: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
# for ex in ds_train.shuffle(10).take(5):
#     print('sequence length', ex[0].shape)

# print('어휘 사전 크기', len(preprocessor.token_counts))

sequence length (10, 810)
sequence length (10, 476)
sequence length (10, 1042)
sequence length (10, 414)
sequence length (10, 600)
어휘 사전 크기 4621


In [16]:
from tensorflow.keras.layers import Embedding
# 임베딩 모델 생성 예시.
model = tf.keras.Sequential()           # 이 임베딩 행렬의 크기는 100 x 6 이다.
model.add(Embedding(input_dim=100,      # 모델이 처리할 단어의 개수, 어휘의 크기
                    output_dim = 6,     # 임베딩 행렬의 차원수
                    input_length=20,
                    name='embed-layer'))
model.build(input_shape=(None, 20))
model.summary() 

/home/jys0714/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embed-layer (Embedding)         │ (None, 20, 6)          │           600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 600 (2.34 KB)

 Trainable params: 600 (2.34 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
token_counts = preprocessor.token_counts
embedding_dim = 20
vocab_size = len(token_counts) + 2 # 아무 단어도 없는 상황에 대한 예약어 0, 현재는 존재하지 않는 단어를 위한 n+1 번째 인덱스 -> 토큰수:n개, 어휘 수:n+2개
tf.random.set_seed(1)

# 모델 생성
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embed-layer'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4, name='lstm-layer')),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
bi_lstm_model.build(input_shape=(None, vocab_size))
bi_lstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embed-layer (Embedding)         │ (None, 4623, 20)       │        92,460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,301 (364.46 KB)

 Trainable params: 93,301 (364.46 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# 컴파일과 훈련
bi_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics=['accuracy'])

history = bi_lstm_model.fit(train_data, validation_data=valid_data, epochs=10)

test_results = bi_lstm_model.evaluate(test_data)
print('test accuracy : {:.2f}%'. format(test_results[1]*100))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.6200 - loss: 0.6930 - val_accuracy: 0.6000 - val_loss: 0.6858
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5950 - loss: 0.6873 - val_accuracy: 0.6000 - val_loss: 0.6801
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5950 - loss: 0.6821 - val_accuracy: 0.6000 - val_loss: 0.6732
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5950 - loss: 0.6751 - val_accuracy: 0.6000 - val_loss: 0.6652
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.5950 - loss: 0.6670 - val_accuracy: 0.6000 - val_loss: 0.6558
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5950 - loss: 0.6576 - val_accuracy: 0.6000 - val_loss: 0.6447
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.5950 - loss: 0.6466 - val_accuracy: 0.6000 - val_loss: 0.6316
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5950 - loss: 0.6334 - val_accuracy: 0.6000 - val_loss: 0.6159

In [ ]:
# target_size = 100

# def divide(train_size=0.4, valid_size = 0.1, test_size=0.5):
    
#     assert train_size + valid_size + test_size == 1.0, "The sum of train, valid and test sizes must be 1.0"
#     train_num = train_size * target_size
#     test_num = test_size * target_size
#     valid_num = valid_size * target_size

#     tf.random.set_seed(1)
#     ds_raw1 = ds_raw.shuffle(target_size, reshuffle_each_iteration=False)

#     ds_raw_test = ds_raw.take(int(train_num))
#     ds_raw_train = ds_raw.take(int(test_num))
#     ds_raw_valid = ds_raw.take(int(valid_num))

#     return ds_raw_train, ds_raw_valid, ds_raw_test

In [ ]:
# ds_raw_train, ds_raw_valid, ds_raw_test = divide()

# for ex in ds_raw_train.take(3):
#     tf.print(ex[0].numpy()[0][:50], ex[1])

In [ ]:
# from collections import Counter
# # 고유 토큰을 수집하는 코드이다.
# tokenizer = tfds.deprecated.text.Tokenizer()
# token_counts = Counter()
# for ex in ds_raw:
#     tokens = tokenizer.tokenize(ex[0].numpy()[0])
#     token_counts.update(tokens)

# print('어휘 사전 크기', len(token_counts))

# # 고유 토큰을 정수로 인코딩하기
# encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
# example_str = 'this is an example!'
# print(encoder.encode(example_str))

# # 텍스트 변환 인코딩 함수 정의 (텍스트_텐서)
# def encode(text_tensor, label):
#     text = text_tensor.numpy()[0]
#     encoded_text = encoder.encode(text)
#     return encoded_text, label

# # 함수를 TF연산으로 변환하기
# def encode_map_fn(text, label):
#     return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

# # 텐서플로 연산을 통해서 데이터셋을 정수리스트로 인코딩했다.
# ds = ds_raw.map(encode_map_fn)


# # ds_train 에서 인코딩된 정수시퀀스 길이 예시 5개 출력
# tf.random.set_seed(1)
# for ex in ds.take(5):
#     print('sequence length', ex[0].shape)

어휘 사전 크기 4621
[95, 10, 170, 4070]
sequence length (114,)
sequence length (226,)
sequence length (162,)
sequence length (83,)
sequence length (171,)
